In [3]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeVectorStore
from pprint import pprint
from langchain.prompts import ChatPromptTemplate





In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini")

index_name = "langchain-index"
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Make actual retriever 
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
    )

In [6]:
template = """Answer the question based only on the following context. Give a one line summary of the context information used at the end of your answer:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [7]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pprint(rag_chain.invoke("What are Biden's tax policies for young adults"))

("The context does not provide specific information about Biden's tax policies "
 'for young adults. It primarily discusses his plans to extend middle-class '
 'tax cuts and not raise taxes on anyone making less than $400,000 per year, '
 'while opposing tax cuts for the wealthy. \n'
 '\n'
 "Summary of context: Biden's tax policies focus on extending middle-class tax "
 'cuts and ensuring the wealthy pay their fair share, without specific mention '
 'of young adults.')
